In [1]:
library(tidyverse)
library(WGCNA)
library(cowplot)
library(matrixStats)

# Custom package
library(rutils)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: dynamicTreeCut
Loading required package: fastcluster

Attaching package: ‘fastcluster’

The following object is masked from ‘package:stats’:

    hclust



Attaching package: ‘WGCNA’

The following object is masked from ‘package:stats’:

    cor


********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************


Attaching package: ‘matrixStats’

The following object is masked 

In [2]:
options(stringsAsFactors = FALSE)
enableWGCNAThreads()

Allowing parallel execution with up to 15 working processes.


In [3]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_path <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

event_code <- list("Alive" = 0, "Dead" = 1)

In [4]:
dset_idx <- 1

# Load data

In [5]:
clinical_df <- load_survival_df(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/survival_data.tsv"), event_code) %>%
    dplyr::select(sample_name, vital_status, survival_time, figo_stage) %>%
    dplyr::filter(rowSums(is.na(.)) == 0)

data_traits <- clinical_df %>%
    decode_figo_stage(to = "n") %>%
    dplyr::mutate(figo_stage = as.character(figo_stage)) %>%
    dplyr::bind_cols(to_one_hot(., col = "figo_stage")) %>%
    dplyr::select(-figo_stage) %>%
    column_to_rownames("sample_name")

Parsed with column specification:
cols(
  sample_name = col_character(),
  vital_status = col_character(),
  survival_time = col_double(),
  figo_stage = col_character(),
  days_to_last_follow_up = col_double(),
  days_to_death = col_double(),
  age_at_diagnosis = col_double(),
  age_at_index = col_double(),
  height = col_double(),
  weight = col_double(),
  bmi = col_double(),
  race = col_character(),
  ethnicity = col_character()
)


In [6]:
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_cancer_network.RData"))

In [7]:
n_genes <- ncol(data_expr)
n_samples <- nrow(data_expr)

In [8]:
mes_0 <- moduleEigengenes(data_expr, module_colors)$eigengenes
mes <- orderMEs(mes_0)

# Module significance for survival analysis

In [9]:
module_coxph_res <- standardScreeningCensoredTime(
    datExpr = mes,
    time = data_traits$survival_time, 
    event = data_traits$vital_status, 
    fastCalculation = FALSE,
    qValues = TRUE
)

In [11]:
module_coxph_res

ID,pvalueWald,qvalueWald,pvalueLogrank,qvalueLogrank,pvalueDeviance,qvalueDeviance,corDeviance,HazardRatio,CI.LowerLimitHR,CI.UpperLimitHR,C.index
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MEblue,0.71172530,0.12347540,0.71165627,0.12346342,0.82953823,0.09627574,0.01356141,2.15624060,0.0366271547,126.937884,0.4892626
MEgreen,0.01240274,0.02151720,0.01203731,0.02088322,0.01069808,0.01365775,0.15941527,103.37668283,2.7252999845,3921.307237,0.3983638
MEpurple,0.45960348,0.08859492,0.45866092,0.08841323,0.50917033,0.07222603,0.04156034,3.95462325,0.1033984418,151.250297,0.4396091
MEmagenta,0.07464291,0.02589921,0.06927315,0.02461806,0.11258111,0.02874542,0.09962103,23.58744985,0.7301943972,761.944754,0.4950574
MEpink,0.44983638,0.08859492,0.44909764,0.08841323,0.46876440,0.07222603,0.04560698,3.92842831,0.1129835192,136.591152,0.4552892
MEred,0.04860866,0.02589921,0.04867081,0.02461806,0.08742551,0.02790303,0.10725144,62.77720324,1.0255278657,3842.876803,0.4382457
MEblack,0.84376026,0.13307437,0.84360954,0.13305060,0.67907814,0.08669477,0.02605610,1.47212758,0.0314590086,68.888364,0.5078968
MEyellow,0.06953267,0.02589921,0.06923378,0.02461806,0.02508277,0.01601101,-0.14018536,0.03187312,0.0007713250,1.317079,0.5965231
MEbrown,0.19783366,0.05720276,0.19758463,0.05713075,0.15583750,0.03315842,-0.08916446,0.06212839,0.0009049679,4.265274,0.5743665


In [12]:
gene_coxph_res <- standardScreeningCensoredTime(
    datExpr = data_expr,
    time = data_traits$survival_time,
    event = data_traits$vital_status,
    fastCalculation = FALSE,
    qValues = TRUE
)

Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”

In [13]:
nrow(gene_coxph_res[gene_coxph_res$qvalueLogrank < 0.05, ])
head(gene_coxph_res[gene_coxph_res$qvalueLogrank < 0.05, ])

[1] 48

,ID,pvalueWald,qvalueWald,pvalueLogrank,qvalueLogrank,pvalueDeviance,qvalueDeviance,corDeviance,HazardRatio,CI.LowerLimitHR,CI.UpperLimitHR,C.index
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14,ANGPT2,0.0004260227,0.01477003,0.0004152209,0.01295850,0.0001682795,0.009779259,0.2326695,1.501484,1.197639,1.882416,0.3325758
50,SERPINA5,0.0029814799,0.04532241,0.0025140092,0.03617566,0.0103938028,0.090000239,0.1600313,1.244038,1.077066,1.436895,0.3789342
53,CSF2,0.0024193091,0.03958573,0.0023151464,0.03569367,0.0034924543,0.053594727,0.1819385,1.361322,1.115319,1.661584,0.3646698
67,P4HA3,0.0003874222,0.01477003,0.0004221192,0.01295850,0.0012626161,0.031446272,0.2003667,1.416678,1.168738,1.717216,0.3996137
74,CXCL2,0.0002679320,0.01249420,0.0002228013,0.01080201,0.0008489514,0.028917575,0.2071277,1.271886,1.117573,1.447505,0.3673446
108,PF4,0.0038271383,0.05570189,0.0033279970,0.04585082,0.0121688436,0.095348793,0.1566391,1.538407,1.148921,2.059930,0.3855965


# Module significance for FIGO stage

In [14]:
module_figo_cor <- WGCNA::cor(mes, data_traits[-c(1:2)])
module_figo_cor

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,-0.130419646,0.020075610,0.17758875,-0.0306603904
MEgreen,-0.069146586,0.039862877,0.07559666,-0.0380222755
MEpurple,0.034966554,0.011484307,-0.05967674,-0.0024735207
MEmagenta,0.034817860,0.001740843,-0.09945942,0.0687326277
MEpink,0.189387367,-0.084598692,-0.14027049,-0.0282833095
MEred,0.169077826,-0.074497461,-0.08466824,-0.0830931269
MEblack,-0.102602181,0.098098551,-0.06480633,0.1254048773
MEyellow,-0.004098473,0.050341150,-0.01586284,-0.0520231763
MEbrown,0.063001885,-0.038066250,-0.02707073,-0.0204201646
MEturquoise,-0.051069808,0.017751013,0.03236273,0.0234145339


In [15]:
module_figo_pvals <- corPvalueStudent(module_figo_cor, n_samples)
module_figo_pvals

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.037406435,0.7496970,0.004447037,0.62603420
MEgreen,0.271299563,0.5262873,0.228986730,0.54557600
MEpurple,0.578349099,0.8551945,0.342556859,0.96864725
MEmagenta,0.579967054,0.9779313,0.113110156,0.27418606
MEpink,0.002389876,0.1780733,0.025089542,0.65305715
MEred,0.006805758,0.2358466,0.177715597,0.18594841
MEblack,0.102113494,0.1181507,0.302599997,0.04543322
MEyellow,0.948073604,0.4234548,0.800978726,0.40811105
MEbrown,0.316289270,0.5451111,0.667021186,0.74554767
MEturquoise,0.416767076,0.7778741,0.606981393,0.70980786


In [16]:
module_figo_padjs <- apply(module_figo_pvals, MARGIN = 2, function(x) { p.adjust(x, method = "BH") })
module_figo_padjs

,figo_stage_1,figo_stage_2,figo_stage_3,figo_stage_4
MEblue,0.13715693,0.9407140,0.04891741,0.9112249
MEgreen,0.57986366,0.8566032,0.50377081,0.9112249
MEpurple,0.64832752,0.9407140,0.53830364,0.9985411
MEmagenta,0.64832752,0.9779313,0.41473724,0.9112249
MEpink,0.02628864,0.6963806,0.13799248,0.9112249
MEred,0.03743167,0.6963806,0.48871789,0.9112249
MEblack,0.28081211,0.6963806,0.53830364,0.4997654
MEyellow,0.94807360,0.8566032,0.80097873,0.9112249
MEbrown,0.57986366,0.8566032,0.73372330,0.9112249
MEturquoise,0.64832752,0.9407140,0.73372330,0.9112249
